In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import get_raw_data
import get_preprocessing_data
import analysis_method
import statsmodels.regression.linear_model as sm

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

pv_list = ["0102", "1701", "1702", "0111", "0330", "0331", "0332", "0333", "0101", "0107", "0110", "5501"]
#["0102"(가구주와의 관계), "1701"(근로소득 유무), "1702"(총 연간근로소득), "0111"(학력졸업여부), "0330"(업종8차),
# "0331"(업종9차), "0332"(직종8차), "0333"(직종9차), "0101"(성별), "0107"(만나이), "0110"(학력학교), "5501"(혼인상태)]
hv_list = ["0150", "1406", "2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155",
           "2156", "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
           "2189", "2190", "2191"]
income_code_list = ["2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155", "2156",
                    "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
                    "2189", "2190", "2191"]

cpi_data = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls").loc[0, :]
cpi_data_index = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls", dtype="str").loc[14, :].values[1:]
cpi_data = pd.Series(cpi_data.values[1:], dtype="float")
cpi_data.name = "CPI"
cpi_data.index = cpi_data_index

industry_8 = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="8차")
industry_large = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="대분류")

industry_return = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\korea_industry_return.xlsx", index_col=0)

data_num = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21"]

cpi_multiple = get_preprocessing_data.get_cpi_multi(cpi_data, data_num)

data_dict = {}
for i in data_num:
    h_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\h_data\\h_" + i + ".pickle"
    p_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\p_data\\p_" + i + ".pickle"

    h_data = pd.read_pickle(h_path)
    p_data = pd.read_pickle(p_path)

    raw_hdata = get_raw_data.get_hv_data(h_data, hv_list, i)
    raw_pdata = get_raw_data.get_pv_data(p_data, pv_list, i)
    
    fin_p_data = get_preprocessing_data.get_fin_personal(raw_pdata, industry_8, industry_large)
    income_data = get_preprocessing_data.get_income_data(fin_p_data.index, raw_hdata)

    fin_x_value = get_preprocessing_data.get_final_xvalue(raw_hdata, fin_p_data)
    fin_y_value = get_preprocessing_data.get_final_yvalue(income_data, cpi_multiple[i])

    finally_preprocessed_data = get_preprocessing_data.get_total_data(fin_y_value, fin_x_value)

    data_dict[i] = finally_preprocessed_data

C:\Users\JeayongYu\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data_df = pd.DataFrame()
for i in data_dict.keys():
    data_df = pd.concat([data_df, data_dict[i]])

_______________________________


### log(Y*)의 계산

In [3]:
temp_edu = data_df.groupby(["0110"]).get_group(1)
temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, 22)
x_mat = analysis_method.get_reg_x(temp_edu).iloc[:, :3]
fer_results = analysis_method.fe_regression_sol(temp_edu, False)

poly_x_mat = analysis_method.polynomial_matrix(temp_edu["0107"], 3) .iloc[:, 1:]
poly_x_mat.index = x_mat.index
fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

In [4]:
fer_results[0][:3]

intercept     7.507209
marriage      0.319783
family_num    0.058957
dtype: float64

In [5]:
fitting_result[1]

intercept    -0.741728
age^1/10^0    0.012367
age^2/10^1    0.006378
age^3/10^2   -0.000899
dtype: float64

In [6]:
fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
fitting_paramters

intercept     6.765481
marriage      0.319783
family_num    0.058957
age^1/10^0    0.012367
age^2/10^1    0.006378
age^3/10^2   -0.000899
dtype: float64

In [7]:
real_data = pd.concat([x_mat, poly_x_mat], axis=1)
real_data

,intercept,marriage,family_num,age^1/10^0,age^2/10^1,age^3/10^2
hhid,,,,,,
21.0,1,1.0,3.0,25.0,62.5,156.25
25.0,1,1.0,4.0,37.0,136.9,506.53
32.0,1,1.0,4.0,59.0,348.1,2053.79
33.0,1,1.0,4.0,42.0,176.4,740.88
40.0,1,1.0,5.0,47.0,220.9,1038.23
...,...,...,...,...,...,...
110085.0,1,1.0,4.0,35.0,122.5,428.75
110102.0,1,1.0,2.0,30.0,90.0,270.00
110122.0,1,1.0,4.0,37.0,136.9,506.53


In [8]:
temp_edu["labor income"]

hhid
21.0        1940.968781
25.0        5822.906343
32.0        2911.453171
33.0        3881.937562
40.0        5822.906343
               ...     
110085.0    6140.000000
110102.0    3500.000000
110122.0    6678.000000
110235.0    6560.000000
110250.0    9700.000000
Name: labor income, Length: 23510, dtype: float64

In [9]:
real_data @ fitting_paramters

hhid
21.0        7.829511
25.0        8.196628
32.0        8.425393
33.0        8.299808
40.0        8.437226
              ...   
110085.0    8.149944
110102.0    7.905564
110122.0    8.196628
110235.0    8.221976
110250.0    8.055704
Length: 23510, dtype: float64

_______________________________


### 3rd-order polynomials vision(각 parameter의 계수를 한번 추정하여 rid계산하는 경우)

In [10]:
fitting_paramters

intercept     6.765481
marriage      0.319783
family_num    0.058957
age^1/10^0    0.012367
age^2/10^1    0.006378
age^3/10^2   -0.000899
dtype: float64

----------------------------------------
### shock calculation

In [11]:
group_sigma_df = pd.DataFrame()
group_pvalue_df = pd.DataFrame()
group_rid_df = pd.DataFrame()

start_list = [22, 24, 26]
for i in range(data_df["0110"].max()):
    df_y_star = pd.DataFrame()

    temp_edu = data_df.groupby(["0110"]).get_group(i+1)
    temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, start_list[i])
    
    fer_results = analysis_method.fe_regression_sol(temp_edu, False)
    fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

    fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
    
    for j in data_num:
        temp_edu_split = data_dict[j].groupby(["0110"]).get_group(i+1)
        temp_edu_split = get_preprocessing_data.get_agecut_data(temp_edu_split, start_list[i])
        
        log_y = np.log(temp_edu_split["labor income"])
        
        x_mat = analysis_method.get_reg_x(temp_edu_split).iloc[:, :3]
        poly_x_mat = analysis_method.polynomial_matrix(temp_edu_split["0107"], 3) .iloc[:, 1:]
        poly_x_mat.index = x_mat.index
        
        temp_real_data = pd.concat([x_mat, poly_x_mat], axis=1)
    
        f_hat = temp_real_data @ fitting_paramters
        
        log_y_star = log_y - f_hat
        df_y_star = pd.concat([df_y_star, log_y_star], axis=1)

    df_y_star.columns = data_num
    
    temp_varrid_list = []
    temp_rid_list = []
    for k in range(len(data_num)-1):
        temp_rid = df_y_star.iloc[:, :-(k+1)].values - df_y_star.iloc[:, k+1:].values
        temp_rid_list.append(temp_rid)
        
        temp_varrid_list.append(np.nanvar(temp_rid))
        
    var_rid_series = pd.Series(temp_varrid_list)
    
    rid_series = (pd.DataFrame(temp_rid_list[0], columns = data_num[1:]) * -1).mean()
    group_rid_df = pd.concat([group_rid_df, rid_series], axis=1)
    
    var_rid_x = np.ones([len(var_rid_series), 2]) * 2
    var_rid_x[:, 0] = var_rid_series.index.values + 1
    var_rid_y = var_rid_series.values
    
    sol = sm.OLS(var_rid_y, var_rid_x)
    sol_result = sol.fit()
    
    sigma_df = pd.DataFrame(sol_result.params, index=["permanent_shock", "idiosyncratic_temporary_shock"], columns=["sigma"])
    pvalue_df = pd.DataFrame(np.round(sol_result.pvalues, 3), index=["permanent_shock(p-value)", "idiosyncratic_temporary_shock(p-value)"], columns=["p-value"])
    
    group_sigma_df = pd.concat([group_sigma_df, sigma_df], axis=1)
    group_pvalue_df = pd.concat([group_pvalue_df, pvalue_df], axis=1)

group_rid_df.columns = ["high_school", "college", "university"]
group_sigma_df.columns = ["high_school", "college", "university"]
group_pvalue_df.columns = ["high_school", "college", "university"]
group_sigma_df = pd.concat([group_sigma_df, group_pvalue_df]).loc[["idiosyncratic_temporary_shock", "permanent_shock", "idiosyncratic_temporary_shock(p-value)", "permanent_shock(p-value)"],:]
group_sigma_df

,high_school,college,university
idiosyncratic_temporary_shock,0.174855,0.145682,0.141262
permanent_shock,0.005461,0.010166,0.005379
idiosyncratic_temporary_shock(p-value),0.000000,0.000000,0.000000
permanent_shock(p-value),0.001000,0.000000,0.009000


----------------------------------------
### Beta & Correlation

In [12]:
ksi = group_rid_df
ksi

,high_school,college,university
02,0.011292,-0.035928,-0.017210
03,0.093463,0.068189,0.046215
04,-0.014766,-0.026539,0.134206
05,0.195853,0.314452,0.109214
06,0.054299,-0.018083,0.038344
07,0.071300,0.107663,0.040838
08,0.030607,0.090565,0.078086
09,0.104878,0.110067,0.075320
10,0.040013,0.111453,0.076239
11,0.032881,-0.001108,0.062704


In [13]:
industry_return

,코스피 음식료품,"코스피 섬유,의복","코스피 종이,목재",코스피 화학,코스피 의약품,코스피 비금속광물,코스피 철강및금속,코스피 기계,"코스피 전기,전자",코스피 운수장비,...,코스피 운수창고,코스피 금융업,코스피 은행,코스피 증권,코스피 보험,코스피 제조업,국고채 총수익 지수,국고채 콜 재투자 지수,국고채 제로 재투자 지수,KRW Cash 총수익 지수
Symbol Name,,,,,,,,,,,,,,,,,,,,,
1,-46.65,-58.69,-54.08,-43.14,-38.08,-59.08,-17.12,-49.55,-26.08,-49.18,...,-53.36,-48.55,-40.12,-51.41,-46.92,-39.22,3.92,3.82,3.65,2.45
2,67.69,19.23,9.75,64.22,7.29,61.33,39.64,21.47,68.55,14.31,...,65.15,9.72,-30.56,254.88,95.04,48.42,3.92,3.82,3.65,2.45
3,68.87,9.13,-16.23,71.33,-0.94,-0.69,53.85,-0.22,153.86,-0.23,...,28.22,-5.43,-6.12,1.08,-9.67,79.95,3.92,3.82,3.65,2.45
4,-51.53,-48.73,-56.78,-50.26,-19.84,-23.47,-35.60,-67.51,-52.89,-35.14,...,-62.62,-46.65,-31.25,-61.47,-39.29,-48.09,3.92,3.82,3.65,2.45
5,45.68,14.90,35.43,51.77,41.50,43.49,49.13,83.24,57.82,64.90,...,10.96,89.52,70.77,117.11,129.13,52.77,3.92,3.82,3.65,2.45
6,-3.45,-42.39,-21.57,-0.75,-19.81,3.96,-0.93,-15.97,5.66,-2.77,...,16.21,-25.48,-13.68,-34.48,3.36,-0.23,3.92,3.82,3.65,2.45
7,43.18,-32.06,4.99,54.72,34.15,51.05,40.67,92.22,38.48,82.50,...,143.98,9.55,18.12,-12.13,14.07,46.09,3.92,3.82,3.65,2.45
8,17.86,-21.87,10.40,35.15,34.43,5.54,17.62,0.40,1.18,2.53,...,20.56,14.81,4.37,1.67,20.50,8.84,3.92,3.82,3.65,2.45
9,58.15,139.07,83.72,26.41,120.20,8.66,16.55,119.94,50.52,91.95,...,42.25,91.51,90.27,193.85,105.50,52.85,-0.74,-0.60,-0.74,2.45


In [14]:
industry_return.index = data_num
industry_ret = industry_return.copy() / 100


beta_dict = {}
for i in ksi.columns:
    beta_list = []
    pvalue_list = []
    eta_list = []
        
    temp_excess_ret = (industry_ret - 0.02)
    for k in industry_ret.columns:
        temp_eta = temp_excess_ret[k] - np.mean(temp_excess_ret)[k]
        eta_list.append(temp_eta)
            
        temp_beta_model = sm.OLS(ksi[i], temp_eta.loc[ksi.index])
        temp_beta_results = temp_beta_model.fit()
            
        beta_list.append(temp_beta_results.params[0])
        pvalue_list.append(temp_beta_results.pvalues[0])
        
    eta = pd.DataFrame(eta_list, index=industry_ret.columns).T
    beta = pd.DataFrame(beta_list, index=industry_ret.columns, columns=["Beta"])
    pvalue = pd.DataFrame(pvalue_list, index=industry_ret.columns, columns=["p-value"])
    
    beta_dict[i] = pd.concat([beta, pvalue], axis=1).T

In [15]:
pd.DataFrame.from_dict({(i,j,k): beta_dict[i][k]
                        for i in beta_dict.keys()
                        for k in beta_dict[i].columns}, orient="index")

Beta   p-value
high_school 21 코스피 음식료품         0.107631  0.027765
               코스피 섬유,의복        0.058077  0.119849
               코스피 종이,목재        0.120719  0.019966
               코스피 화학           0.089400  0.043233
               코스피 의약품          0.066478  0.120734
...                                  ...       ...
university  21 코스피 제조업          0.012331  0.765716
               국고채 총수익 지수      -0.353949  0.514775
               국고채 콜 재투자 지수    -0.358365  0.533056
               국고채 제로 재투자 지수   -0.371646  0.526812
               KRW Cash 총수익 지수  0.358484  0.909600

[66 rows x 2 columns]

In [16]:
corr_df = pd.DataFrame()
for i in [0, 1, 2]:
        
    corr_list = []
    for k in eta.columns:
        corr_list.append(np.corrcoef(eta.loc[ksi.index, :][k], ksi[ksi.columns[i]])[0, 1])
        
    temp_corr_df = pd.DataFrame(corr_list, index=eta.columns)
    corr_df = pd.concat([corr_df, temp_corr_df], axis=1)
    
corr_df.columns = ksi.columns
corr_df

,high_school,college,university
코스피 음식료품,0.591524,0.393545,-0.113672
"코스피 섬유,의복",0.426809,0.225440,0.007968
"코스피 종이,목재",0.630353,0.469569,0.107650
코스피 화학,0.549329,0.304489,-0.001785
코스피 의약품,0.426488,0.380393,0.163423
코스피 비금속광물,0.393044,0.253829,-0.102394
코스피 철강및금속,0.488872,0.349346,0.150218
코스피 기계,0.634303,0.530158,0.217843
"코스피 전기,전자",0.559443,0.260791,-0.127370
코스피 운수장비,0.649245,0.386054,0.275259


In [17]:
corr_mat_dic = {}
for i in ksi.columns:
    corr_mat_dic[i] = pd.concat([ksi[i], eta], axis=1).dropna().corr()
    
sample_corr_mat = pd.DataFrame.from_dict({(i,k): corr_mat_dic[i][k]
                            for i in corr_mat_dic.keys()
                            for k in corr_mat_dic[i].columns}, orient="index")
sample_corr_mat

high_school  코스피 음식료품  코스피 섬유,의복  코스피 종이,목재  \
high_school high_school         1.000000  0.591524   0.426809   0.630353   
            코스피 음식료품            0.591524  1.000000   0.502944   0.625081   
            코스피 섬유,의복           0.426809  0.502944   1.000000   0.752954   
            코스피 종이,목재           0.630353  0.625081   0.752954   1.000000   
            코스피 화학              0.549329  0.780795   0.289830   0.540036   
...                                  ...       ...        ...        ...   
university  코스피 제조업                  NaN  0.800365   0.471717   0.603833   
            국고채 총수익 지수               NaN -0.292422  -0.443387  -0.530869   
            국고채 콜 재투자 지수             NaN -0.289415  -0.443902  -0.529844   
            국고채 제로 재투자 지수            NaN -0.287716  -0.445539  -0.533523   
            KRW Cash 총수익 지수          NaN  0.250373   0.112560   0.087479   

                               코스피 화학   코스피 의약품  코스피 비금속광물  코스피 철강및금속  \
high_school high_school      0.549329  0.426488   0.393044   0.488872   
            코스피 음식료품         0.780795  0.537273   0.539343   0.509478   
            코스피 섬유,의복        0.289830  0.571091   0.394454   0.274943   
            코스피 종이,목재        0.540036  0.761125   0.486341   0.405147   
            코스피 화학           1.000000  0.414614   0.544188   0.748850   
...                               ...       ...        ...        ...   
university  코스피 제조업          0.881512  0.426863   0.502509   0.793286   
            국고채 총수익 지수      -0.288148 -0.600180  -0.305075  -0.386709   
            국고채 콜 재투자 지수    -0.285475 -0.607272  -0.309397  -0.376711   
            국고채 제로 재투자 지수   -0.280710 -0.610171  -0.310432  -0.377991   
            KRW Cash 총수익 지수 -0.081040 -0.112504   0.066342  -0.192849   

                               코스피 기계  코스피 전기,전자  ...    코스피 은행    코스피 증권  \
high_school high_school      0.634303   0.559443  ...  0.696850  0.344230   
            코스피 음식료품         0.599818   0.697766  ...  0.311084  0.657259   
            코스피 섬유,의복        0.540378   0.343219  ...  0.624090  0.619021   
            코스피 종이,목재        0.798870   0.342363  ...  0.695489  0.676487   
            코스피 화학           0.685789   0.671276  ...  0.387660  0.538927   
...                               ...        ...  ...       ...       ...   
university  코스피 제조업          0.725161   0.895334  ...  0.616894  0.598280   
            국고채 총수익 지수      -0.460245  -0.381035  ... -0.601292 -0.385217   
            국고채 콜 재투자 지수    -0.455611  -0.381417  ... -0.603889 -0.385085   
            국고채 제로 재투자 지수   -0.457754  -0.376306  ... -0.602888 -0.387320   
            KRW Cash 총수익 지수 -0.120970  -0.122136  ... -0.197995 -0.046106   

                               코스피 보험   코스피 제조업  국고채 총수익 지수  국고채 콜 재투자 지수  \
high_school high_school      0.616793  0.687369   -0.427237     -0.421843   
            코스피 음식료품         0.632542  0.800365   -0.292422     -0.289415   
            코스피 섬유,의복        0.569847  0.471717   -0.443387     -0.443902   
            코스피 종이,목재        0.771391  0.603833   -0.530869     -0.529844   
            코스피 화학           0.570633  0.881512   -0.288148     -0.285475   
...                               ...       ...         ...           ...   
university  코스피 제조업          0.612234  1.000000   -0.469219     -0.466254   
            국고채 총수익 지수      -0.439971 -0.469219    1.000000      0.999557   
            국고채 콜 재투자 지수    -0.434969 -0.466254    0.999557      1.000000   
            국고채 제로 재투자 지수   -0.437660 -0.461079    0.999054      0.999576   
            KRW Cash 총수익 지수 -0.008032 -0.113761    0.364692      0.376334   

                             국고채 제로 재투자 지수  KRW Cash 총수익 지수  college  \
high_school high_school          -0.419559         0.068825      NaN   
            코스피 음식료품             -0.287716         0.250373      NaN   
            코스피 섬유,의복            -0.445539         0.112560      NaN   
            코스피 종이,목재            -0.533523         0.087479      NaN   
            코스피 화학           

In [18]:
corr_mat_dic["university"]

,university,코스피 음식료품,"코스피 섬유,의복","코스피 종이,목재",코스피 화학,코스피 의약품,코스피 비금속광물,코스피 철강및금속,코스피 기계,"코스피 전기,전자",...,코스피 운수창고,코스피 금융업,코스피 은행,코스피 증권,코스피 보험,코스피 제조업,국고채 총수익 지수,국고채 콜 재투자 지수,국고채 제로 재투자 지수,KRW Cash 총수익 지수
university,1.000000,-0.113672,0.007968,0.107650,-0.001785,0.163423,-0.102394,0.150218,0.217843,-0.127370,...,-0.062001,0.287943,0.299454,-0.055465,0.199370,0.004658,-0.239588,-0.229239,-0.232850,0.041914
코스피 음식료품,-0.113672,1.000000,0.502944,0.625081,0.780795,0.537273,0.539343,0.509478,0.599818,0.697766,...,0.568218,0.548694,0.311084,0.657259,0.632542,0.800365,-0.292422,-0.289415,-0.287716,0.250373
"코스피 섬유,의복",0.007968,0.502944,1.000000,0.752954,0.289830,0.571091,0.394454,0.274943,0.540378,0.343219,...,0.228236,0.695350,0.624090,0.619021,0.569847,0.471717,-0.443387,-0.443902,-0.445539,0.112560
"코스피 종이,목재",0.107650,0.625081,0.752954,1.000000,0.540036,0.761125,0.486341,0.405147,0.798870,0.342363,...,0.420955,0.863988,0.695489,0.676487,0.771391,0.603833,-0.530869,-0.529844,-0.533523,0.087479
코스피 화학,-0.001785,0.780795,0.289830,0.540036,1.000000,0.414614,0.544188,0.748850,0.685789,0.671276,...,0.687785,0.552212,0.387660,0.538927,0.570633,0.881512,-0.288148,-0.285475,-0.280710,-0.081040
코스피 의약품,0.163423,0.537273,0.571091,0.761125,0.414614,1.000000,0.283691,0.197248,0.647193,0.207711,...,0.261780,0.686961,0.551951,0.505959,0.561843,0.426863,-0.600180,-0.607272,-0.610171,-0.112504
코스피 비금속광물,-0.102394,0.539343,0.394454,0.486341,0.544188,0.283691,1.000000,0.483326,0.521303,0.323604,...,0.659339,0.484085,0.362926,0.558129,0.594350,0.502509,-0.305075,-0.309397,-0.310432,0.066342
코스피 철강및금속,0.150218,0.509478,0.274943,0.405147,0.748850,0.197248,0.483326,1.000000,0.672620,0.634665,...,0.649943,0.583880,0.517369,0.457708,0.589287,0.793286,-0.386709,-0.376711,-0.377991,-0.192849
코스피 기계,0.217843,0.599818,0.540378,0.798870,0.685789,0.647193,0.521303,0.672620,1.000000,0.393844,...,0.747472,0.803523,0.649778,0.629489,0.790977,0.725161,-0.460245,-0.455611,-0.457754,-0.120970
"코스피 전기,전자",-0.127370,0.697766,0.343219,0.342363,0.671276,0.207711,0.323604,0.634665,0.393844,1.000000,...,0.378773,0.500595,0.482981,0.439152,0.356797,0.895334,-0.381035,-0.381417,-0.376306,-0.122136


In [19]:
corr_mat_dic["university"].to_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\university_corr.xlsx")